In [1]:
import requests
import pandas as pd

# URL de base para a API de livros
base_url = "https://openlibrary.org/search.json"

# Parametros da requisição de acordo com documentação da API e necessidade do usuário
params = {
    "q": "harry potter OR lords of the rings OR jane austen",
    "fields": "title, publish_year, author_key, author_name, subject",
    "sort": "rating",
    "limit": 1000
}

# Faz a requisição GET para a API com os parâmetros definidos
response = requests.get(base_url, params=params)

if response.status_code == 200: # Verifica se a requisição foi bem-sucedida, cód 200 padrão HTTP
    data = response.json() # Converte a resposta JSON em um dicionário Python
    livros = data["docs"] # Cria objeto livros que contém apenas os dados que estão dentro da chave docs da resposta da API.
    df_livros = pd.DataFrame(livros) # Cria um DataFrame pandas a partir da lista de livros
    print(df_livros.head()) # Exibe as primeiras linhas do DataFrame com as colunas especificadas no fields
else:
    print(f"Erro na requisição: {response.status_code}")

print(response.status_code)
print(response.text)

                                          author_key  \
0                                         [OL19981A]   
1                                         [OL53336A]   
2                                         [OL23186A]   
3  [OL115574A, OL2760531A, OL10790687A, OL13002189A]   
4                                         [OL25930A]   

                                         author_name  \
0                                     [Stephen King]   
1                               [Desiderius Erasmus]   
2                      [William Makepeace Thackeray]   
3  [Abbé Prévost, F. C. Green, felix crespo, Pr&e...   
4                                 [Charles Perrault]   

                                        publish_year  \
0  [1996, 1997, 1998, 1999, 2000, 2001, 2002, 200...   
1  [1536, 1540, 1549, 1557, 1560, 1617, 1636, 164...   
2  [1800, 1847, 1848, 1849, 1850, 1865, 1867, 187...   
3  [1734, 1753, 1787, 1800, 1839, 1840, 1841, 184...   
4  [1675, 1697, 1723, 1729, 1741, 1742, 1764, 

Tratamento dos dados

1. Entendendo os dados

In [2]:
df_livros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   author_key    24 non-null     object
 1   author_name   24 non-null     object
 2   publish_year  25 non-null     object
 3   title         25 non-null     object
 4   subject       19 non-null     object
dtypes: object(5)
memory usage: 1.1+ KB


In [3]:
df_livros.columns

Index(['author_key', 'author_name', 'publish_year', 'title', 'subject'], dtype='object')

In [4]:
df_livros.shape

(25, 5)

In [5]:
df_livros.isnull().sum()

author_key      1
author_name     1
publish_year    0
title           0
subject         6
dtype: int64

2. Normalizando os dados  
2.1 Padronizando nulos

In [6]:
print(df_livros[df_livros['subject'].isnull()])

                  author_key                                 author_name  \
10              [OL3454345A]                          [Chapin A. Harris]   
14  [OL6010190A, OL9349345A]          [Chapin Aaron Harris, P H. Austen]   
15              [OL2511721A]                            [Fransham, John]   
16              [OL4289649A]                              [Ralph Harris]   
17              [OL4339229A]  [Harris of High Cross, Ralph Harris Baron]   
20               [OL983260A]                              [Ralph Harris]   

    publish_year                                              title subject  
10  [1875, 1895]           The principles and practice of dentistry     NaN  
14        [2018]           The Principles and Practice of Dentistry     NaN  
15  [1740, 1741]  The world in miniature: or, the entertaining t...     NaN  
16        [1956]                         Politics without prejudice     NaN  
17        [1956]                         Politics without prejudice     NaN  

In [7]:
df_livros['author_name'].fillna('Sem informação', inplace=True) # Preenche valores nulos com 'Sem informação'
df_livros.isnull().sum()

C:\Users\basil\AppData\Local\Temp\ipykernel_39048\2781930646.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_livros['author_name'].fillna('Sem informação', inplace=True) # Preenche valores nulos com 'Sem informação'


author_key      1
author_name     0
publish_year    0
title           0
subject         6
dtype: int64

In [8]:
df_livros['subject'].fillna('Sem informação', inplace=True) # Preenche valores nulos com 'Sem informação'
df_livros.isnull().sum()

C:\Users\basil\AppData\Local\Temp\ipykernel_39048\795722335.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_livros['subject'].fillna('Sem informação', inplace=True) # Preenche valores nulos com 'Sem informação'


author_key      1
author_name     0
publish_year    0
title           0
subject         0
dtype: int64

2.2 Padronizando author_name

In [9]:
# Verifica os tipos de dados na coluna author_name
df_livros['author_name'].apply(type).value_counts()

author_name
<class 'list'>    24
<class 'str'>      1
Name: count, dtype: int64

In [10]:
# Padroniza os nomes dos autores removendo acentuação e convertendo para minúsculas
!pip install unidecode
from unidecode import unidecode

def padronizando_autor(autor): # Função chamada de 'padronizando_autor' para padronizar o parâmetro 'autor'
    return unidecode(autor. lower()) # Função retorna o parâmetro 'autor' sem acentuação e em letras minúsculas

# Cria uma nova coluna 'autor_padronizado' aplicando a função 'padronizando_autor' em cada elemento da coluna 'author_name'
df_livros['autor_padronizado'] = df_livros['author_name'].apply(lambda x: [padronizando_autor(autor) for autor in x] if isinstance(x, list) else [])
df_livros.head()


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


,author_key,author_name,publish_year,title,subject,autor_padronizado
0,[OL19981A],[Stephen King],"[1996, 1997, 1998, 1999, 2000, 2001, 2002, 200...",The Green Mile,"[horror fiction, fiction, death row inmates, p...",[stephen king]
1,[OL53336A],[Desiderius Erasmus],"[1536, 1540, 1549, 1557, 1560, 1617, 1636, 164...",Moriæ Encomium,"[Aspectos religiosos, Bibliography, Catalogs, ...",[desiderius erasmus]
2,[OL23186A],[William Makepeace Thackeray],"[1800, 1847, 1848, 1849, 1850, 1865, 1867, 187...",Vanity Fair,"[Social life and customs, fiction, Women, fict...",[william makepeace thackeray]
3,"[OL115574A, OL2760531A, OL10790687A, OL13002189A]","[Abbé Prévost, F. C. Green, felix crespo, Pr&e...","[1734, 1753, 1787, 1800, 1839, 1840, 1841, 184...",Manon Lescaut,"[Fiction, History, France in fiction, French l...","[abbe prevost, f. c. green, felix crespo, pr&e..."
4,[OL25930A],[Charles Perrault],"[1675, 1697, 1723, 1729, 1741, 1742, 1764, 178...",Les Contes de ma mère l'Oye,"[Nursery rhymes, France, Children's stories, F...",[charles perrault]


2.2.1 Criando lista de autores únicos

In [11]:
# Lista criada para incluir apenas os autores que estão em formato lista dentro do df_livros
lista_autores = df_livros['autor_padronizado'][df_livros['autor_padronizado'].apply(lambda x: isinstance(x, list))]

print(lista_autores[:5])

0                                       [stephen king]
1                                 [desiderius erasmus]
2                        [william makepeace thackeray]
3    [abbe prevost, f. c. green, felix crespo, pr&e...
4                                   [charles perrault]
Name: autor_padronizado, dtype: object


In [12]:
# Transformando a série de listas em uma lista contínua de nomes de autores
lista_autores_continua = [] # Lista vazia para armazenar os autores
for lista in lista_autores: # Para cada lista presente na lista_autores
    lista_autores_continua.extend(lista) # Adicionar cada autor da lista à lista_autores_continua

print(lista_autores_continua[:5])

['stephen king', 'desiderius erasmus', 'william makepeace thackeray', 'abbe prevost', 'f. c. green']


In [13]:
# Cria um DataFrame com os autores únicos, ordenados alfabeticamente
df_autores_unicos = pd.DataFrame(sorted(set(lista_autores_continua)), columns=['autor'])
df_autores_unicos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   autor   23 non-null     object
dtypes: object(1)
memory usage: 316.0+ bytes


In [14]:
# Verifica os tipos de dados na coluna autor do DataFrame df_autores_unicos
df_autores_unicos['autor'].apply(type).value_counts()

autor
<class 'str'>    23
Name: count, dtype: int64

In [15]:
df_livros.head()

,author_key,author_name,publish_year,title,subject,autor_padronizado
0,[OL19981A],[Stephen King],"[1996, 1997, 1998, 1999, 2000, 2001, 2002, 200...",The Green Mile,"[horror fiction, fiction, death row inmates, p...",[stephen king]
1,[OL53336A],[Desiderius Erasmus],"[1536, 1540, 1549, 1557, 1560, 1617, 1636, 164...",Moriæ Encomium,"[Aspectos religiosos, Bibliography, Catalogs, ...",[desiderius erasmus]
2,[OL23186A],[William Makepeace Thackeray],"[1800, 1847, 1848, 1849, 1850, 1865, 1867, 187...",Vanity Fair,"[Social life and customs, fiction, Women, fict...",[william makepeace thackeray]
3,"[OL115574A, OL2760531A, OL10790687A, OL13002189A]","[Abbé Prévost, F. C. Green, felix crespo, Pr&e...","[1734, 1753, 1787, 1800, 1839, 1840, 1841, 184...",Manon Lescaut,"[Fiction, History, France in fiction, French l...","[abbe prevost, f. c. green, felix crespo, pr&e..."
4,[OL25930A],[Charles Perrault],"[1675, 1697, 1723, 1729, 1741, 1742, 1764, 178...",Les Contes de ma mère l'Oye,"[Nursery rhymes, France, Children's stories, F...",[charles perrault]


2.3 Padronizando subtitle

In [16]:
def padronizando_assunto(assunto): # Função chamada de 'padronizando_assunto' para padronizar o parâmetro 'assunto'
    return unidecode(assunto.lower()) # Função retorna o parâmetro 'assunto' sem acentuação e em letras minúsculas


# Cria uma nova coluna 'assunto_padronizado' aplicando a função 'padronizando_assunto' em cada elemento da coluna 'subject'
df_livros['assunto_padronizado'] = df_livros[ 'subject'].apply(lambda x: [padronizando_assunto(assunto) for assunto in x] if isinstance(x, list) else [])
df_livros.head()

,author_key,author_name,publish_year,title,subject,autor_padronizado,assunto_padronizado
0,[OL19981A],[Stephen King],"[1996, 1997, 1998, 1999, 2000, 2001, 2002, 200...",The Green Mile,"[horror fiction, fiction, death row inmates, p...",[stephen king],"[horror fiction, fiction, death row inmates, p..."
1,[OL53336A],[Desiderius Erasmus],"[1536, 1540, 1549, 1557, 1560, 1617, 1636, 164...",Moriæ Encomium,"[Aspectos religiosos, Bibliography, Catalogs, ...",[desiderius erasmus],"[aspectos religiosos, bibliography, catalogs, ..."
2,[OL23186A],[William Makepeace Thackeray],"[1800, 1847, 1848, 1849, 1850, 1865, 1867, 187...",Vanity Fair,"[Social life and customs, fiction, Women, fict...",[william makepeace thackeray],"[social life and customs, fiction, women, fict..."
3,"[OL115574A, OL2760531A, OL10790687A, OL13002189A]","[Abbé Prévost, F. C. Green, felix crespo, Pr&e...","[1734, 1753, 1787, 1800, 1839, 1840, 1841, 184...",Manon Lescaut,"[Fiction, History, France in fiction, French l...","[abbe prevost, f. c. green, felix crespo, pr&e...","[fiction, history, france in fiction, french l..."
4,[OL25930A],[Charles Perrault],"[1675, 1697, 1723, 1729, 1741, 1742, 1764, 178...",Les Contes de ma mère l'Oye,"[Nursery rhymes, France, Children's stories, F...",[charles perrault],"[nursery rhymes, france, children's stories, f..."


2.3.1 Criando lista de assuntos únicos

In [17]:
# Verifica se há algum valor que não seja lista
nao_lista = df_livros['assunto_padronizado'].apply(lambda x: not isinstance(x, list))

# Se houver pelo menos um valor que não seja lista, mostra as linhas
if nao_lista.any():
    print("Existem valores que não são listas.")
    print(df_livros.loc[nao_lista, 'assunto_padronizado'].head())
else:
    print("Todos os valores são listas.")

Todos os valores são listas.


In [18]:
# Lista criada para incluir apenas os assuntos que estão em formato lista dentro do df_livros
lista_assuntos = df_livros['assunto_padronizado'][df_livros['assunto_padronizado'].apply(lambda x: isinstance(x, list))]
print(lista_assuntos[:5])


0    [horror fiction, fiction, death row inmates, p...
1    [aspectos religiosos, bibliography, catalogs, ...
2    [social life and customs, fiction, women, fict...
3    [fiction, history, france in fiction, french l...
4    [nursery rhymes, france, children's stories, f...
Name: assunto_padronizado, dtype: object


In [19]:
lista_assuntos_continua = [] # Lista vazia para armazenar os assuntos
for lista in lista_assuntos: # Para cada lista presente na lista_assuntos
    lista_assuntos_continua.extend(lista) # Adicionar cada assunto da lista à lista_assuntos_continua

print(lista_assuntos_continua[:5])

['horror fiction', 'fiction', 'death row inmates', 'prisons', 'executions and executioners']


In [20]:
df_assuntos_unicos = pd.DataFrame(sorted(set(lista_assuntos_continua)), columns=['assunto'])
df_assuntos_unicos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   assunto  286 non-null    object
dtypes: object(1)
memory usage: 2.4+ KB


In [21]:
df_assuntos_unicos['assunto'].apply(type).value_counts()

assunto
<class 'str'>    286
Name: count, dtype: int64

2.4 Padronizando publish_year

In [22]:
lista_anos = df_livros['publish_year'][df_livros['publish_year'].apply(lambda x: isinstance(x, list))]
print(lista_anos[:5])

0    [1996, 1997, 1998, 1999, 2000, 2001, 2002, 200...
1    [1536, 1540, 1549, 1557, 1560, 1617, 1636, 164...
2    [1800, 1847, 1848, 1849, 1850, 1865, 1867, 187...
3    [1734, 1753, 1787, 1800, 1839, 1840, 1841, 184...
4    [1675, 1697, 1723, 1729, 1741, 1742, 1764, 178...
Name: publish_year, dtype: object


In [23]:
lista_anos_continua = [] # Lista vazia para armazenar os anos
for lista in lista_anos: # Para cada lista presente na lista_anos
    lista_anos_continua.extend(lista) # Adicionar cada ano da lista à lista_anos_continua

print(lista_anos_continua[:5])

[1996, 1997, 1998, 1999, 2000]


In [24]:
df_anos_unicos = pd.DataFrame(sorted((set(lista_anos_continua))), columns=['ano'])
df_anos_unicos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   ano     276 non-null    int64
dtypes: int64(1)
memory usage: 2.3 KB


In [25]:
df_anos_unicos['ano'].apply(type).value_counts()

ano
<class 'int'>    276
Name: count, dtype: int64

2.5 Padronizando title

In [26]:
def padronizando_titulo(titulo):
    return unidecode(titulo.lower())

df_livros['titulo_padronizado'] = df_livros['title'].apply(padronizando_titulo)
df_livros.head()

,author_key,author_name,publish_year,title,subject,autor_padronizado,assunto_padronizado,titulo_padronizado
0,[OL19981A],[Stephen King],"[1996, 1997, 1998, 1999, 2000, 2001, 2002, 200...",The Green Mile,"[horror fiction, fiction, death row inmates, p...",[stephen king],"[horror fiction, fiction, death row inmates, p...",the green mile
1,[OL53336A],[Desiderius Erasmus],"[1536, 1540, 1549, 1557, 1560, 1617, 1636, 164...",Moriæ Encomium,"[Aspectos religiosos, Bibliography, Catalogs, ...",[desiderius erasmus],"[aspectos religiosos, bibliography, catalogs, ...",moriae encomium
2,[OL23186A],[William Makepeace Thackeray],"[1800, 1847, 1848, 1849, 1850, 1865, 1867, 187...",Vanity Fair,"[Social life and customs, fiction, Women, fict...",[william makepeace thackeray],"[social life and customs, fiction, women, fict...",vanity fair
3,"[OL115574A, OL2760531A, OL10790687A, OL13002189A]","[Abbé Prévost, F. C. Green, felix crespo, Pr&e...","[1734, 1753, 1787, 1800, 1839, 1840, 1841, 184...",Manon Lescaut,"[Fiction, History, France in fiction, French l...","[abbe prevost, f. c. green, felix crespo, pr&e...","[fiction, history, france in fiction, french l...",manon lescaut
4,[OL25930A],[Charles Perrault],"[1675, 1697, 1723, 1729, 1741, 1742, 1764, 178...",Les Contes de ma mère l'Oye,"[Nursery rhymes, France, Children's stories, F...",[charles perrault],"[nursery rhymes, france, children's stories, f...",les contes de ma mere l'oye


In [27]:
df_livros['titulo_padronizado'].apply(type).value_counts()

titulo_padronizado
<class 'str'>    25
Name: count, dtype: int64

In [28]:
df_titulos_unicos = pd.DataFrame(sorted(set(df_livros['titulo_padronizado'])), columns=['titulo'])
df_titulos_unicos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   titulo  19 non-null     object
dtypes: object(1)
memory usage: 284.0+ bytes


Transformando listas em registros únicos por linha (autor_padronizado, publish_year e assunto_padronizado)

In [29]:
df_livros['assunto_padronizado'].apply(lambda x: isinstance(x, list)).value_counts()

assunto_padronizado
True    25
Name: count, dtype: int64

In [30]:
df_livros['publish_year'].apply(lambda x: isinstance(x, list)).value_counts()

publish_year
True    25
Name: count, dtype: int64

In [31]:
df_livros['autor_padronizado'].apply(lambda x: isinstance(x, list)).value_counts()

autor_padronizado
True    25
Name: count, dtype: int64

In [37]:
df_livros_exploded = df_livros.explode('autor_padronizado')
df_livros_exploded = df_livros_exploded.explode('publish_year')
df_livros_exploded = df_livros_exploded.explode('assunto_padronizado')
df_livros_exploded.head()

,author_key,author_name,publish_year,title,subject,autor_padronizado,assunto_padronizado,titulo_padronizado
0,[OL19981A],[Stephen King],1996,The Green Mile,"[horror fiction, fiction, death row inmates, p...",stephen king,horror fiction,the green mile
0,[OL19981A],[Stephen King],1996,The Green Mile,"[horror fiction, fiction, death row inmates, p...",stephen king,fiction,the green mile
0,[OL19981A],[Stephen King],1996,The Green Mile,"[horror fiction, fiction, death row inmates, p...",stephen king,death row inmates,the green mile
0,[OL19981A],[Stephen King],1996,The Green Mile,"[horror fiction, fiction, death row inmates, p...",stephen king,prisons,the green mile
0,[OL19981A],[Stephen King],1996,The Green Mile,"[horror fiction, fiction, death row inmates, p...",stephen king,executions and executioners,the green mile


Removendo colunas que não serão utilizadas no sql

In [39]:
print(df_livros_exploded.columns.tolist())


['author_key', 'author_name', 'publish_year', 'title', 'subject', 'autor_padronizado', 'assunto_padronizado', 'titulo_padronizado']


In [40]:
df_livros_exploded = df_livros_exploded.drop(columns=['author_key', 'author_name', 'title', 'subject'])

In [41]:
print(df_livros_exploded.columns.tolist())

['publish_year', 'autor_padronizado', 'assunto_padronizado', 'titulo_padronizado']


In [42]:
df_livros_exploded = df_livros_exploded[['titulo_padronizado', 'autor_padronizado', 'assunto_padronizado', 'publish_year', ]]

In [43]:
df_livros_exploded = df_livros_exploded.rename(columns={'publish_year': 'ano_publicacao'})
df_livros_exploded.head()

,titulo_padronizado,autor_padronizado,assunto_padronizado,ano_publicacao
0,the green mile,stephen king,horror fiction,1996
0,the green mile,stephen king,fiction,1996
0,the green mile,stephen king,death row inmates,1996
0,the green mile,stephen king,prisons,1996
0,the green mile,stephen king,executions and executioners,1996


In [45]:
print(df_livros_exploded.columns.tolist())

['titulo_padronizado', 'autor_padronizado', 'assunto_padronizado', 'ano_publicacao']


2.6 Transformar listas do df_livros em str

In [ ]:
# Convertendo listas em strings separadas por vírgulas para facilitar a visualização e mantendo as demais colunas inalteradas
for col in df_livros.columns:
    if df_livros[col].apply(lambda x: isinstance(x, list)).any():
        df_livros[col] = df_livros[col].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
df_livros.head()

3. Conexão com MySQL

In [ ]:
!pip install sqlalchemy

from dotenv import load_dotenv
import os
from sqlalchemy import create_engine

# Carrega as variáveis do .env
load_dotenv()

# Lê as variáveis
usuario = os.getenv('MYSQL_USUARIO')
senha = os.getenv('MYSQL_SENHA')
host = os.getenv('MYSQL_HOST')
porta = os.getenv('MYSQL_PORTA')
banco = os.getenv('MYSQL_BANCO')

print("Usuário:", usuario)
print("Senha:", senha)
print("Host:", host)
print("Porta:", porta)
print("Banco:", banco)



In [ ]:
# Cria a conexão
!pip install pymysql



In [ ]:
engine = create_engine(f'mysql+pymysql://{usuario}:{senha}@{host}:{porta}/{banco}')

In [ ]:
    # Exporta df_livros
    df_livros.to_sql(name='livros', con=engine, if_exists='replace', index=False)

    # Exporta df_titulos_unicos
    df_titulos_unicos.to_sql(name='titulos_unicos', con=engine, if_exists='replace', index=False)

    # Exporta df_autores_unicos
    df_autores_unicos.to_sql(name='autores_unicos', con=engine, if_exists='replace', index=False)

    # Exporta df_anos_unicos
    df_anos_unicos.to_sql(name='anos_unicos', con=engine, if_exists='replace', index=False)

    # Exporta df_assuntos_unicos
    df_assuntos_unicos.to_sql(name='assuntos_unicos', con=engine, if_exists='replace', index=False)


    print("Todos os DataFrames foram exportados com sucesso para o MySQL!")